This notebook contains examples of using model index and opc ua functions to download inverters, strings set and trackers data of the site 'EG-AS' from the model index and opc ua api servers.

### Import Libraries

In [ ]:
%load_ext autotime

In [ ]:
# Import the required packeages
import pandas as pd
import urllib3
urllib3.disable_warnings()
from pandas.io.json import json_normalize
from typing import List, Dict
import sys
import os
import numpy as np
import concurrent.futures
import json
import itertools
from aiohttp import ClientSession
import asyncio
import dask.dataframe as dd

### Import Scripts

In [ ]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

In [ ]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opc_url = "http://10.241.68.86:13371/"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(url=opc_url)

### OPC UA server parameters

In [ ]:
# Input parameters for value data
server_url = "opc.tcp://10.241.80.4:4872"
# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z"  
end_time = "2022-07-06T07:55:14.544000Z" # 5th July aggregated data
end_time1 = "2022-07-20T07:55:14.544000Z" # 15 days
end_time2 = "2022-08-05T07:55:14.544000Z" # 30 days
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

In [ ]:
# All the sites on the OPC server
sites = mdx.get_objects_of_type('SiteType')

# Selecting specific site 
sitename = 'EG-AS'
site = sites[sites['DisplayName'] == sitename]

INVERTERS DATA

In [ ]:
# All the inverters on the site
inverters = mdx.get_object_descendants("InverterType", site, "PV_Assets")

In [ ]:
# List of unique inverter variables
inv_var_list = ['ACActivePower', 'ACCurrentAverage', 'ACCurrentPhaseA', 'ACCurrentPhaseB', 'ACCurrentPhaseC', 'ACDailyActiveEnergy', 'ACReactivePower', 'ACTotalActiveEnergy', 
'ACVoltagePhaseAB', 'ACVoltagePhaseAC', 'ACVoltagePhaseBC', 'DCCurrent', 'DCPower', 'DCVoltage', 'Frequency', 'PowerFactor', 'Status']

In [ ]:
# Live value data of trackers 
live_value = opc.get_live_values_data(server_url, ['ACActivePower', 'ACCurrentAverage'], inverters)
live_value

In [ ]:
# Live value inverters data
inv_liv_values = opc.get_live_values_data(server_url, inv_var_list, inverters)
inv_liv_values

In [ ]:
# 1 day aggregated historical inverter data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

In [ ]:
# 15 day inverters data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time1, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

In [ ]:
# 1 month inverter data 
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time2, pro_interval, agg_name, inverters, inv_var_list)
await session.close()

STRINGS DATA

In [ ]:
# Strings set data 
strings = mdx.get_object_descendants("StringSetType", site, "PV_Assets")

In [ ]:
# List of strings variables
strng_var_list = ['DCCurrent', 'DCPower', 'DCVoltage', 'RelativeDCPower', 'Status']

In [ ]:
# Live value strings data 
strng_liv_values = opc.get_live_values_data(server_url, strng_var_list, strings)

In [ ]:
# 1 day strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time, pro_interval, agg_name, strings, strng_var_list)
await session.close()

In [ ]:
# 15 days strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time1, pro_interval, agg_name, strings, strng_var_list)
await session.close()

In [ ]:
# 1 month strings data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time2, pro_interval, agg_name, strings, strng_var_list)
await session.close()

TRACKERS DATA

In [ ]:
# Trackers data 
trackers = mdx.get_object_ancestors("TrackerType", strings, "PV_Serves")

In [ ]:
track_var_list = ['AngleMeasured', 'AngleSetpoint']

In [ ]:
# Live value trackers data
track_liv_values = opc.get_live_values_data(server_url, track_var_list, trackers)

In [ ]:
# 1 day trackers data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time, pro_interval, agg_name, trackers, track_var_list)
await session.close()

In [ ]:
# 10 day trackers data 
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time1, pro_interval, agg_name, trackers, track_var_list)
await session.close()

In [ ]:
# 1 month trackers data 
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time2, pro_interval, agg_name, trackers, track_var_list)
await session.close()